# Tower Demand Curve for One Renewal Period

## Overview
Find the optimal insurance tower structure — deductible (retention) and maximum limit — across
a range of loss ratios to formulate a demand curve for insurance.  The optimizer maximizes Sharpe ratio, subject to a hard constraint of no more than 0.25% probability of ruin over one renewal year.

The tower is built from fixed layer breakpoints but the optimizer can "squish" it from both ends by choosing where to start (deductible) and where to stop (max limit).

- **Prerequisites**: [optimization/01_optimization_overview](01_optimization_overview.ipynb)
- **Audience**: [Practitioner] / [Developer]

In [1]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/optimization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Setup complete. If you see numpy/scipy import errors below,
restart the runtime (Runtime > Restart runtime) and re-run all cells.


## Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import multiprocessing

warnings.filterwarnings("ignore")

from ergodic_insurance.pareto_frontier import (
    Objective, ObjectiveType, ParetoFrontier, ParetoPoint,
)
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.insurance_program import (
    EnhancedInsuranceLayer, InsuranceProgram,
)
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

plt.style.use("seaborn-v0_8-darkgrid")
SEED = 42
np.random.seed(SEED)
N_CORES = multiprocessing.cpu_count()
print(f"Number of CPU cores: {N_CORES}")  # Available parallel cores for sensitivity sweep
CI = False      # Set True to skip heavy computations

Number of CPU cores: 44


## Part I: Parameter Setup

### Manufacturing Company Configuration

Baseline company parameters (the experiment varies revenue via `revenue_grid`).

In [3]:
# --- Economic Parameters
ATR = 2.0                # Asset turnover ratio
OPERATING_MARGIN = 0.15  # 12% EBIT margin before Insurable Losses
REV_VOL = 0.50           # Revenue volatility (annualized)
INITIAL_ASSETS = 5_000_000

# --- Company Configuration ---
mfg_config = ManufacturerConfig(
    initial_assets=INITIAL_ASSETS,          # $15M total assets
    asset_turnover_ratio=ATR,               # Revenue = Assets Ãƒâ€” turnover = $22.5M
    base_operating_margin=OPERATING_MARGIN, # 12% EBIT margin -> $2.7M/yr operating income
    tax_rate=0.25,                          # 25% corporate tax
    retention_ratio=0.70,                   # 70% earnings retained for growth
)

# Display company profile
revenue = mfg_config.initial_assets * mfg_config.asset_turnover_ratio
ebit = revenue * mfg_config.base_operating_margin
print("=" * 60)
print("MANUFACTURING COMPANY PROFILE")
print("=" * 60)
print(f"Total Assets:          ${mfg_config.initial_assets:>14,.0f}")
print(f"Annual Revenue:        ${revenue:>14,.0f}")
print(f"Operating Income:      ${ebit:>14,.0f}")
print(f"Operating Margin:      {mfg_config.base_operating_margin:>14.1%}")
print(f"Asset Turnover:        {mfg_config.asset_turnover_ratio:>14.1f}x")
print(f"Revenue Volatility:    {REV_VOL:>14}")
print(f"Tax Rate:              {mfg_config.tax_rate:>13.1%}")
print(f"Retention Ratio:       {mfg_config.retention_ratio:>13.1%}")
print("=" * 60)

MANUFACTURING COMPANY PROFILE
Total Assets:          $     5,000,000
Annual Revenue:        $    10,000,000
Operating Income:      $     1,500,000
Operating Margin:               15.0%
Asset Turnover:                   2.0x
Revenue Volatility:               0.5
Tax Rate:                      25.0%
Retention Ratio:               70.0%


### Shared Simulation Infrastructure

Loss model, analytical LEV-based layer pricing, CRN scenario generation, and simulation engine.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from concurrent.futures import ProcessPoolExecutor
import warnings
import logging
import time

# Suppress all warnings and verbose solver logging
warnings.filterwarnings("ignore")
logging.getLogger("ergodic_insurance").setLevel(logging.ERROR)

from ergodic_insurance.hjb_solver import (
    StateVariable, ControlVariable, StateSpace,
    LogUtility, PowerUtility, ExpectedWealth,
    HJBProblem, HJBSolver, HJBSolverConfig,
)
from ergodic_insurance.optimal_control import (
    ControlSpace, StaticControl, HJBFeedbackControl,
    TimeVaryingControl, OptimalController,
)
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.insurance_program import (
    EnhancedInsuranceLayer, InsuranceProgram,
)
from ergodic_insurance.loss_distributions import (
    ManufacturingLossGenerator, LognormalLoss, ParetoLoss,
)
from ergodic_insurance.insurance_pricing import LayerPricer

plt.style.use("seaborn-v0_8-darkgrid")
SEED = 42
np.random.seed(SEED)
N_CORES = 40   # Available parallel cores for sensitivity sweep
CI = False      # Set True to skip heavy computations

# =====================================================
# SHARED SIMULATION INFRASTRUCTURE
# =====================================================
# Used by Parts 5, 8, 9, 10, and 11.

# --- Economic Parameters ---
REFERENCE_REVENUE = ATR * INITIAL_ASSETS  # Fixed reference for loss calibration

# --- Loss Scaling ---
# Loss frequency (and CRN loss amounts) scale with the square root of
# revenue.  This keeps the loss drag proportional to the company's
# actual size for both insured and uninsured strategies.
FREQ_SCALING_EXPONENT = 0.75

# --- Amplified Loss Model ---
ATTR_BASE_FREQ = 3
ATTR_SEV_MEAN = 10_000
ATTR_SEV_CV = 15

LG_BASE_FREQ = 1.25
LG_SEV_MEAN = 400_000
LG_SEV_CV = 7

CAT_BASE_FREQ = 0.15
CAT_SEV_ALPHA = 2.01
CAT_SEV_XM = 1_000_000

LOSS_PARAMS = dict(
    attritional_params={'base_frequency': ATTR_BASE_FREQ,
                        'severity_mean': ATTR_SEV_MEAN,
                        'severity_cv': ATTR_SEV_CV,
                        'revenue_scaling_exponent': FREQ_SCALING_EXPONENT,
                        'reference_revenue': REFERENCE_REVENUE},
    large_params={'base_frequency': LG_BASE_FREQ,
                  'severity_mean': LG_SEV_MEAN,
                  'severity_cv': LG_SEV_CV,
                  'revenue_scaling_exponent': FREQ_SCALING_EXPONENT,
                  'reference_revenue': REFERENCE_REVENUE},
    catastrophic_params={'base_frequency': CAT_BASE_FREQ,
                         'severity_alpha': CAT_SEV_ALPHA,
                         'severity_xm': CAT_SEV_XM,
                         'revenue_scaling_exponent': FREQ_SCALING_EXPONENT,
                         'reference_revenue': REFERENCE_REVENUE},
)

# Quick validation of the loss model
_val_gen = ManufacturingLossGenerator(**LOSS_PARAMS, seed=99)
_val_totals = []
SCENARIOS = 10_000
for _ in range(SCENARIOS):
    _events, _stats = _val_gen.generate_losses(duration=1.0, revenue=REFERENCE_REVENUE)
    _val_totals.append(_stats['total_amount'])
_expected_annual_loss = np.mean(_val_totals)
_operating_income = INITIAL_ASSETS * ATR * OPERATING_MARGIN
print(f"Loss model validation ({SCENARIOS:,.0f} one-year samples):")
print(f"  Expected annual loss:  ${_expected_annual_loss:>12,.0f}")
print(f"  Operating income:      ${_operating_income:>12,.0f}")
print(f"  Loss / Income ratio:   {_expected_annual_loss / _operating_income:.0%}")
print(f"  Std dev annual loss:   ${np.std(_val_totals):>12,.0f}")
print(f"  Max annual loss:       ${np.max(_val_totals):>12,.0f}")
del _val_gen, _val_totals, _events, _stats


# --- Analytical Layer Pricing via LEV ---
# Instead of hardcoded rate-on-line values, we compute actuarially sound
# premiums from the known severity distributions using limited expected
# values (LEVs).  For each layer (attachment a, limit l):
#
#   E[layer loss] = sum_i  freq_i * [LEV_i(a+l) - LEV_i(a)]
#   premium       = E[layer loss] / target_loss_ratio
#   rate_on_line  = premium / limit
#
# This ensures the primary-layer ROL decreases naturally as the Ded
# (retention) rises, producing the genuine cost-vs-variance tradeoff
# that the HJB solver needs.
#
# The pricers are parameterized so that the sensitivity analysis (Part 9)
# can adapt premiums to match the modified loss assumptions being tested.

TARGET_LOSS_RATIO = 0.7  # Normal-market loss ratio
LOSS_RATIO_INFLECTION = 1  # Factor by which the max layer loss ratio differs from the base

def make_layer_pricers(large_freq=LG_BASE_FREQ,
                        large_sev_mean=LG_SEV_MEAN,
                        cur_revenue=REFERENCE_REVENUE) -> tuple:
    """Create a tuple of LayerPricers for a given loss parameterization.

    Frequency scales as (revenue / reference)^0.5, matching the loss
    model's sub-linear revenue scaling.  This keeps premium and loss
    scaling consistent so that insured and uninsured strategies face
    the same proportional cost growth.

    Args:
        large_freq: Large-loss annual frequency (default 1.0).
        large_sev_mean: Large-loss mean severity (default $1M).
        cur_revenue: Current revenue for frequency scaling.

    Returns:
        Tuple of (attritional, large, catastrophic) LayerPricers.
    """
    scale = (cur_revenue / REFERENCE_REVENUE) ** FREQ_SCALING_EXPONENT
    return (
        LayerPricer(LognormalLoss(mean=ATTR_SEV_MEAN, cv=ATTR_SEV_CV),
                    frequency=ATTR_BASE_FREQ * scale),
        LayerPricer(LognormalLoss(mean=large_sev_mean, cv=LG_SEV_CV),
                    frequency=large_freq * scale),
        LayerPricer(ParetoLoss(alpha=CAT_SEV_ALPHA, xm=CAT_SEV_XM),
                    frequency=CAT_BASE_FREQ * scale),
    )


# Default pricers for baseline loss model
DEFAULT_PRICERS = make_layer_pricers()

MIN_LAYER_MIDPOINT = np.mean([0, 5_000_000])
MAX_LAYER_MIDPOINT = np.mean([450_000_000, 500_000_000])

def analytical_layer_premium(attachment: float, limit: float,
                            base_loss_ratio: float,
                            loss_ratio_inflection: float,
                            pricers=None) -> float:
    """Compute actuarial premium for a layer using LEV-based expected losses.

    Premium = E[layer loss] / layer_loss_ratio, where:
      E[layer loss] = sum over components of freq_i * (LEV_i(a+l) - LEV_i(a))

    Args:
        attachment: Layer attachment point.
        limit: Layer limit (width of coverage).
        pricers: Tuple of LayerPricers. Uses DEFAULT_PRICERS if None.
        base_loss_ratio: Desired loss ratio for the layer.
        loss_ratio_inflection: Factor by which the max layer loss ratio  differs from the base.
    """

    pricers = pricers or DEFAULT_PRICERS
    expected_loss = sum(p.expected_layer_loss(attachment, limit) for p in pricers)
    cur_layer_midpoint = np.mean([attachment, attachment + limit])
    layer_loss_ratio = base_loss_ratio + \
                        (1.0 / loss_ratio_inflection - 1.0) * base_loss_ratio \
                        * (cur_layer_midpoint - MIN_LAYER_MIDPOINT) \
                        / (MAX_LAYER_MIDPOINT - MIN_LAYER_MIDPOINT)
    return expected_loss / layer_loss_ratio


def analytical_rate_on_line(attachment: float, limit: float,
                            base_loss_ratio: float,
                            loss_ratio_inflection: float,
                            pricers=None) -> float:
    """Compute rate-on-line for a layer: premium / limit."""
    if limit <= 0:
        return 0.0
    return analytical_layer_premium(attachment,
                                    limit,
                                    base_loss_ratio,
                                    loss_ratio_inflection,
                                    pricers) / limit


# Validate: show how ROL varies across sample attachment points
print(f"\nAnalytical layer pricing (target LR = {TARGET_LOSS_RATIO:.0%}):")
print(f"  {'Attachment':>12s}  {'Limit':>12s}  {'E[Loss]':>12s}  {'Premium':>12s}  {'ROL':>8s}")
print(f"  {'-'*12}  {'-'*12}  {'-'*12}  {'-'*12}  {'-'*8}")
for _a, _l in [(10_000, 4_990_000), (25_000, 4_975_000), (50_000, 4_950_000),
                (250_000, 4_750_000), (1_000_000, 4_000_000),
                (2_000_000, 3_000_000), (4_000_000, 1_000_000),
                (5_000_000, 20_000_000), (25_000_000, 25_000_000), (50_000_000, 50_000_000)]:
    _el = analytical_layer_premium(_a, _l, TARGET_LOSS_RATIO, 1.0) * TARGET_LOSS_RATIO
    _p = analytical_layer_premium(_a, _l, TARGET_LOSS_RATIO, 1.0)
    _r = analytical_rate_on_line(_a, _l, TARGET_LOSS_RATIO, 1.0)
    print(f"  ${_a:>11,.0f}  ${_l:>11,.0f}  ${_el:>11,.0f}  ${_p:>11,.0f}  {_r:>7.2%}")


# --- Insurance Tower Factory ---
# Premium rates are computed analytically from the loss distribution,
# ensuring that the primary-layer ROL decreases with higher retention.
# The optional `pricers` argument lets the sensitivity analysis pass
# in LayerPricers built from alternative loss assumptions, so that
# premiums stay consistent with the loss environment being tested.

def make_program(ded: float,
                base_loss_ratio: float,
                loss_ratio_inflection: float,
                max_limit: float,
                pricers=None) -> InsuranceProgram:
    """Create 4-layer tower with analytically priced premiums.

    Uses LEV-based layer pricing from severity distributions so that
    rate-on-line adjusts naturally with the retention level.

    Args:
        ded: Deductible.
        max_limit: Maximum coverage limit (top of tower).
        pricers: Tuple of LayerPricers. Uses DEFAULT_PRICERS if None.

    Returns:
        InsuranceProgram with actuarially sound premium loading.
    """
    layer_defs = [
        # (attachment, ceiling, reinstatements)
        (0, 5_000_000, 0),
        (5_000_000, 10_000_000, 0),
        (10_000_000, 25_000_000, 0),
        (25_000_000, 50_000_000, 0),
        (50_000_000, 100_000_000, 0),
        (100_000_000, 150_000_000, 0),
        (150_000_000, 200_000_000, 0),
        (200_000_000, 250_000_000, 0),
        (250_000_000, 300_000_000, 0),
        (300_000_000, 350_000_000, 0),
        (350_000_000, 400_000_000, 0),
        (400_000_000, 450_000_000, 0),
        (450_000_000, 500_000_000, 0),
    ]
    layers = []
    for attach, ceiling, reinst in layer_defs:
        if ded >= ceiling:
            continue  # Skip layers that are fully below the deductible
        if max_limit is not None and ceiling > max_limit:
            continue  # Skip layers that exceed the max limit constraint
        # Now we're within the working layer
        # The deductible is below, the max limit is above
        effective_attach = max(attach, ded)
        limit = ceiling - effective_attach
        if limit <= 0:
            continue
        rol = analytical_rate_on_line(effective_attach, limit, base_loss_ratio, loss_ratio_inflection, pricers)
        layers.append(EnhancedInsuranceLayer(
            attachment_point=effective_attach,
            limit=limit,
            base_premium_rate=rol,
            reinstatements=reinst,
        ))
    return InsuranceProgram(
        layers=layers,
        deductible=ded,
        name=f"Manufacturing Tower (Ded=${ded:,.0f})",
    )


# --- CRN: Pre-generate Loss Scenarios ---
def generate_loss_pool(n_paths, n_years, reference_revenue=REFERENCE_REVENUE, seed=SEED, specific_loss_params=None):
    """Pre-generate loss scenarios for Common Random Number comparison.
    All strategies will face the exact same loss events and revenue shocks.
    Losses are generated at a fixed reference revenue; the simulation
    engine then scales event amounts by (actual_revenue / reference)^0.5
    so that loss burden grows proportionally with the company.

    Args:
        n_paths: Number of simulation paths.
        n_years: Number of years to simulate.
        reference_revenue: Reference revenue for loss calibration.
        seed: Base seed for random number generation.
        specific_loss_params: A dictionary of loss parameters to use, overriding
                              the global LOSS_PARAMS for this call.
    """
    loss_params_to_use = specific_loss_params if specific_loss_params is not None else LOSS_PARAMS

    ss = np.random.SeedSequence(seed)
    children = ss.spawn(n_paths + 1)

    # Shared revenue shocks
    rev_rng = np.random.default_rng(children[0])
    revenue_shocks = rev_rng.standard_normal((n_paths, n_years))

    # Per-path loss event sequences
    all_losses = []  # [path][year] -> List[LossEvent]
    for i in range(n_paths):
        gen = ManufacturingLossGenerator(
            **loss_params_to_use, # Use the potentially overridden loss params
            seed=int(children[i + 1].generate_state(1)[0] % (2**31)),
        )
        path_losses = []
        for t in range(n_years):
            events, _ = gen.generate_losses(duration=1.0, revenue=reference_revenue)
            path_losses.append(events)
        all_losses.append(path_losses)

    return revenue_shocks, all_losses


# --- CRN Simulation Engine ---
def simulate_with_crn(ded,
                    base_loss_ratio: float,
                    loss_ratio_inflection: float,
                    max_limit: float,
                    revenue_shocks, loss_pool, n_years=1,
                    initial_assets=INITIAL_ASSETS, pricers=None):
    """Simulate one static-Ded strategy across all CRN paths.

    Uses the library's InsuranceProgram.process_claim() to correctly
    allocate each loss through the insurance tower.

    Loss amounts from the CRN pool are scaled by
    (actual_revenue / REFERENCE_REVENUE)^FREQ_SCALING_EXPONENT so that
    the loss burden grows proportionally with the company.  Premium is
    repriced at actual revenue with the same exponent, keeping the
    cost-of-risk consistent between insured and uninsured strategies.

    Args:
        ded: Deductible.
        revenue_shocks: Pre-generated revenue shocks (n_paths x n_years).
        loss_pool: Pre-generated loss events [path][year] -> List[LossEvent].
        n_years: Simulation horizon.
        initial_assets: Starting wealth.
        pricers: Tuple of LayerPricers for premium calculation.
            Uses DEFAULT_PRICERS if None (baseline loss assumptions).

    Returns:
        paths: array of shape (n_paths, n_years + 1) with asset values.
    """
    n_paths = len(loss_pool)
    paths = np.zeros((n_paths, n_years + 1))
    paths[:, 0] = initial_assets

    # Build program template and get fixed annual premium
    if ded >= 100_000_000:
        # "No insurance" -- skip tower entirely
        annual_premium = 0.0
        use_insurance = False
        program_template = None
    else:
        program_template = make_program(ded,
                                        base_loss_ratio,
                                        loss_ratio_inflection,
                                        max_limit,
                                        pricers=pricers)
        annual_premium = program_template.calculate_premium()
        use_insurance = True

    for i in range(n_paths):
        assets = initial_assets
        for t in range(n_years):
            # Operating income with shared revenue shock
            revenue = assets * ATR * np.exp(
                REV_VOL * revenue_shocks[i, t] - 0.5 * REV_VOL**2
            )
            operating_income = revenue * OPERATING_MARGIN

            # Scale CRN losses to current revenue (sqrt scaling)
            loss_scale = (revenue / REFERENCE_REVENUE) ** FREQ_SCALING_EXPONENT

            # Process losses through insurance tower
            total_retained = 0.0
            if use_insurance:
                new_pricers = make_layer_pricers(cur_revenue=revenue)
                program_update = make_program(ded,
                                                base_loss_ratio,
                                                loss_ratio_inflection,
                                                max_limit,
                                                pricers=new_pricers)
                annual_premium = program_update.calculate_premium()
                program = InsuranceProgram.create_fresh(program_update)
                for event in loss_pool[i][t]:
                    scaled_amount = event.amount * loss_scale
                    result = program.process_claim(scaled_amount)
                    total_retained += result.deductible_paid + result.uncovered_loss
            else:
                for event in loss_pool[i][t]:
                    total_retained += event.amount * loss_scale

            # Net income and asset update
            assets = assets + operating_income - total_retained - annual_premium
            assets = max(assets, 0.0)
            paths[i, t + 1] = assets

    return paths


# Pre-generate the main CRN pool
N_PATHS = 500
N_YEARS = 1
print(f"\nPre-generating CRN loss pool ({N_PATHS:,.0f} paths x {N_YEARS} years)...")
t0 = time.time()
CRN_SHOCKS, CRN_LOSSES = generate_loss_pool(n_paths=N_PATHS, n_years=N_YEARS)
print(f"  Done in {time.time() - t0:.1f}s")
print(f"  Shape: {CRN_SHOCKS.shape[0]:,} paths x {CRN_SHOCKS.shape[1]} years")

# Quick sanity: total losses per path-year
_annual_totals = [
    sum(e.amount for e in CRN_LOSSES[i][t])
    for i in range(N_PATHS) for t in range(N_YEARS)
]
print(f"  Mean annual loss: ${np.mean(_annual_totals):,.0f}")
del _annual_totals


Loss model validation (10,000 one-year samples):
  Expected annual loss:  $     810,152
  Operating income:      $   1,500,000
  Loss / Income ratio:   54%
  Std dev annual loss:   $   2,232,722
  Max annual loss:       $  78,067,932

Analytical layer pricing (target LR = 70%):
    Attachment         Limit       E[Loss]       Premium       ROL
  ------------  ------------  ------------  ------------  --------
  $     10,000  $  4,990,000  $    658,191  $    940,273   18.84%
  $     25,000  $  4,975,000  $    638,549  $    912,213   18.34%
  $     50,000  $  4,950,000  $    613,275  $    876,107   17.70%
  $    250,000  $  4,750,000  $    493,355  $    704,793   14.84%
  $  1,000,000  $  4,000,000  $    260,635  $    372,335    9.31%
  $  2,000,000  $  3,000,000  $    121,171  $    173,101    5.77%
  $  4,000,000  $  1,000,000  $     24,540  $     35,057    3.51%
  $  5,000,000  $ 20,000,000  $    109,088  $    155,841    0.78%
  $ 25,000,000  $ 25,000,000  $     20,511  $     29,301   

In [5]:
# =====================================================
# SOBOL QUASI-RANDOM LOSS GENERATION & IMPORTANCE SAMPLING
# =====================================================
# Quasi-random low-discrepancy sequences for variance reduction,
# plus rejection-based importance sampling for tail events.

from scipy.stats.qmc import Sobol as _Sobol
from scipy.stats import norm as _sp_norm, poisson as _sp_poisson

# Truncation limits for compound Poisson event counts
MAX_ATTR_EVENTS  = 10
MAX_LARGE_EVENTS = 6
MAX_CAT_EVENTS   = 3
MAX_EVENTS = MAX_ATTR_EVENTS + MAX_LARGE_EVENTS + MAX_CAT_EVENTS   # 19
SOBOL_DIMS = 1 + (1 + MAX_ATTR_EVENTS) + (1 + MAX_LARGE_EVENTS) + (1 + MAX_CAT_EVENTS)  # 23


def generate_sobol_loss_pool(n_paths, revenue, seed=0):
    """Generate loss scenarios using 23-dim Sobol quasi-random sequences.

    Dimension layout:
      0:       revenue shock  ~ N(0,1)
      1-11:    attritional count (Poisson) + 10 severities (Lognormal)
      12-18:   large count (Poisson) + 6 severities (Lognormal)
      19-22:   cat count (Poisson) + 3 severities (Pareto)

    Returns:
        revenue_shocks:       (n_paths, 1) standard normal shocks
        loss_amounts:         (n_paths, MAX_EVENTS) individual losses (0 = unused slot)
        max_individual_loss:  (n_paths,) largest single loss per path
    """
    scale = (revenue / REFERENCE_REVENUE) ** FREQ_SCALING_EXPONENT
    attr_freq = ATTR_BASE_FREQ * scale
    lg_freq   = LG_BASE_FREQ * scale
    cat_freq  = CAT_BASE_FREQ * scale

    attr_var = np.log(1 + ATTR_SEV_CV**2)
    attr_mu  = np.log(ATTR_SEV_MEAN) - attr_var / 2
    attr_sig = np.sqrt(attr_var)

    lg_var = np.log(1 + LG_SEV_CV**2)
    lg_mu  = np.log(LG_SEV_MEAN) - lg_var / 2
    lg_sig = np.sqrt(lg_var)

    sampler = _Sobol(d=SOBOL_DIMS, scramble=True, seed=seed)
    m = int(np.ceil(np.log2(max(n_paths, 2))))
    U = sampler.random(2**m)[:n_paths]
    eps = 1e-10
    col = 0

    # Revenue shock ~ N(0,1)
    revenue_shocks = _sp_norm.ppf(np.clip(U[:, col], eps, 1 - eps)).reshape(-1, 1)
    col += 1

    # Attritional: Poisson count + Lognormal severities
    attr_n = _sp_poisson.ppf(np.clip(U[:, col], 0, 1 - eps), attr_freq).astype(int)
    attr_n = np.minimum(attr_n, MAX_ATTR_EVENTS)
    col += 1
    attr_raw = np.exp(attr_mu + attr_sig * _sp_norm.ppf(
        np.clip(U[:, col:col + MAX_ATTR_EVENTS], eps, 1 - eps)))
    col += MAX_ATTR_EVENTS
    attr_sevs = attr_raw * (np.arange(MAX_ATTR_EVENTS) < attr_n[:, None])

    # Large: Poisson count + Lognormal severities
    lg_n = _sp_poisson.ppf(np.clip(U[:, col], 0, 1 - eps), lg_freq).astype(int)
    lg_n = np.minimum(lg_n, MAX_LARGE_EVENTS)
    col += 1
    lg_raw = np.exp(lg_mu + lg_sig * _sp_norm.ppf(
        np.clip(U[:, col:col + MAX_LARGE_EVENTS], eps, 1 - eps)))
    col += MAX_LARGE_EVENTS
    lg_sevs = lg_raw * (np.arange(MAX_LARGE_EVENTS) < lg_n[:, None])

    # Catastrophic: Poisson count + Pareto severities
    cat_n = _sp_poisson.ppf(np.clip(U[:, col], 0, 1 - eps), cat_freq).astype(int)
    cat_n = np.minimum(cat_n, MAX_CAT_EVENTS)
    col += 1
    cat_raw = CAT_SEV_XM / (1 - np.clip(
        U[:, col:col + MAX_CAT_EVENTS], 0, 1 - eps)) ** (1 / CAT_SEV_ALPHA)
    col += MAX_CAT_EVENTS
    cat_sevs = cat_raw * (np.arange(MAX_CAT_EVENTS) < cat_n[:, None])

    loss_amounts = np.hstack([attr_sevs, lg_sevs, cat_sevs])
    max_individual_loss = np.max(loss_amounts, axis=1)

    return revenue_shocks, loss_amounts, max_individual_loss


def generate_is_pool_rejection(n_target, revenue, threshold=50_000_000,
                               seed_offset=1_000_000, batch_size=260_000,
                               verbose=False):
    """Generate importance-sampled tail paths via rejection sampling.

    Generates Sobol batches, keeps only paths where the max individual
    loss exceeds the threshold.  Continues until n_target paths collected.

    Returns:
        is_shocks:  (n_target, 1) revenue shocks for tail paths
        is_losses:  (n_target, MAX_EVENTS) loss amounts for tail paths
        p_tail:     estimated tail probability (acceptance rate)
    """
    collected_shocks, collected_losses = [], []
    n_generated = n_accepted = batch_num = 0

    while n_accepted < n_target:
        shocks, losses, max_loss = generate_sobol_loss_pool(
            batch_size, revenue, seed=seed_offset + batch_num)
        mask = max_loss > threshold
        n_hit = int(mask.sum())
        if n_hit > 0:
            collected_shocks.append(shocks[mask])
            collected_losses.append(losses[mask])
            n_accepted += n_hit
        n_generated += len(max_loss)
        batch_num += 1
        if verbose and batch_num % 10 == 0:
            print(f"  IS rejection: {n_accepted}/{n_target} from "
                  f"{n_generated:,} ({n_accepted/max(n_generated,1):.6%})")

    is_shocks = np.vstack(collected_shocks)[:n_target]
    is_losses = np.vstack(collected_losses)[:n_target]
    p_tail = n_accepted / n_generated

    if verbose:
        print(f"  IS complete: {n_target} tail paths from {n_generated:,} "
              f"({p_tail:.6%}, {batch_num} batches)")

    return is_shocks, is_losses, p_tail


def compute_stratified_stats(W_T_main, W_T_is, tail_mask_main, initial_assets):
    """Combine main and IS pools via stratified estimation.

    The tail probability is estimated from the main pool (unbiased).
    Tail-stratum statistics use both main-pool tail paths and IS paths
    for improved precision in the extreme-loss regime.

    Returns:
        (growth_rate, growth_vol, ruin_prob)
    """
    N_main = len(W_T_main)
    p_tail = tail_mask_main.sum() / N_main

    log_g_main = np.log(np.maximum(W_T_main, 1.0) / initial_assets)
    log_g_is   = np.log(np.maximum(W_T_is, 1.0) / initial_assets)

    normal_mask = ~tail_mask_main
    if normal_mask.any():
        lg_n = log_g_main[normal_mask]
        mean_n, meansq_n = lg_n.mean(), (lg_n**2).mean()
        ruin_n = float((W_T_main[normal_mask] <= 0).mean())
    else:
        mean_n = meansq_n = ruin_n = 0.0

    # Tail stratum: main-pool tail paths + IS paths
    if tail_mask_main.any():
        lg_t = np.concatenate([log_g_main[tail_mask_main], log_g_is])
        wt_t = np.concatenate([W_T_main[tail_mask_main], W_T_is])
    else:
        lg_t, wt_t = log_g_is, W_T_is

    mean_t, meansq_t = lg_t.mean(), (lg_t**2).mean()
    ruin_t = float((wt_t <= 0).mean())

    growth_rate = (1 - p_tail) * mean_n + p_tail * mean_t
    E_sq = (1 - p_tail) * meansq_n + p_tail * meansq_t
    growth_vol  = np.sqrt(max(E_sq - growth_rate**2, 0.0))
    ruin_prob   = (1 - p_tail) * ruin_n + p_tail * ruin_t

    return growth_rate, growth_vol, ruin_prob


# Quick validation of the Sobol generator
_val_shocks, _val_losses, _val_max = generate_sobol_loss_pool(10_000, REFERENCE_REVENUE, seed=99)
_val_totals = _val_losses.sum(axis=1)
print(f"\nSobol loss pool validation (10,000 paths at reference revenue):")
print(f"  Mean annual loss:   ${np.mean(_val_totals):>12,.0f}")
print(f"  Std dev:            ${np.std(_val_totals):>12,.0f}")
print(f"  Max single event:   ${np.max(_val_max):>12,.0f}")
print(f"  Paths with loss > $50M event: {(np.max(_val_losses, axis=1) > 50_000_000).sum()}")
print(f"  Sobol dims: {SOBOL_DIMS}, Max events/path: {MAX_EVENTS}")
del _val_shocks, _val_losses, _val_max, _val_totals


Sobol loss pool validation (10,000 paths at reference revenue):
  Mean annual loss:   $     835,429
  Std dev:            $   2,902,205
  Max single event:   $ 141,541,691
  Paths with loss > $50M event: 4
  Sobol dims: 23, Max events/path: 19


## Part II: Insurance Demand Curve

For each revenue level in `revenue_grid`, we sweep all valid `(ded, hollow_bottom, hollow_top, max_limit)` tuples
and simulate `SENS_N_PATHS` one-year CRN paths.

The **hollow tower** has coverage in two bands with a gap:
- **Lower band**: `[ded, hollow_bottom]` — primary / working layer coverage
- **Hollow (gap)**: `[hollow_bottom, hollow_top]` — **no coverage** (retained by the company)
- **Upper band**: `[hollow_top, max_limit]` — excess layer coverage

The optimal tower at each revenue is selected by maximizing a **weighted objective**:
- **75% weight** on annual ergodic growth rate
- **25% weight** on reducing result volatility (std of log-growth)

subject to the **hard constraint** that the probability of ruin does not exceed **0.5%**.

Losses scale with revenue via a **0.75 exponent**, and the optimizer chooses where to place the
deductible (bottom), hollow section (gap boundaries), and max limit (top).

In [6]:
import itertools

# --- HOLLOW TOWER GRIDS ---
# 4D sweep: (ded, hollow_bottom, hollow_top, max_limit)
# Coverage: [ded, hollow_bottom] + [hollow_top, max_limit]
# Gap:      [hollow_bottom, hollow_top]  (no coverage)
#
# Grids are coarser than the non-hollow experiment to keep the
# combinatorial explosion manageable.  Expand for production runs.

all_ranges = np.geomspace(10_000, 1_000_000_000, 40)

ded_grid = list(filter(lambda x: x < 250_000_000, all_ranges))

hollow_grid = list(filter(lambda x: (x > 100_000 and x < 650_000_000), all_ranges))

max_limit_grid = list(filter(lambda x: x > 500_000, all_ranges))

### 3 outer parameters to sweep: revenue, base loss ratio, inflection factor
base_loss_ratio_grid = np.linspace(0.4, 0.9, 11)

inflection_factor_grid = np.array([
    1.0,
    2.0,
    3.0,
])

revenue_grid = np.array([
    5_000_000,
    25_000_000,
    50_000_000,
])

# Grid for parallel sweeping to determine demand curves
scan_grid = list(itertools.product(revenue_grid, base_loss_ratio_grid, inflection_factor_grid))

# Count valid 4-tuples (ded < hb < ht < ml)
n_valid = 0
for d in ded_grid:
    for i_hb, hb in enumerate(hollow_grid):
        if hb < d:
            continue
        for ht in hollow_grid[i_hb + 1:]:
            for ml in max_limit_grid:
                if ml < ht:
                    continue
                n_valid += 1

print(f"Deductible grid:   {len(ded_grid)} values "
      f"(${ded_grid[0]:,.0f} -- ${ded_grid[-1]:,.0f})")
print(f"Hollow grid:       {len(hollow_grid)} values "
      f"(${hollow_grid[0]:,.0f} -- ${hollow_grid[-1]:,.0f})")
print(f"Max limit grid:    {len(max_limit_grid)} values "
      f"(${max_limit_grid[0]:,.0f} -- ${max_limit_grid[-1]:,.0f})")
print(f"Revenue grid:      {len(revenue_grid)} values "
      f"(${revenue_grid[0]:,.0f} -- ${revenue_grid[-1]:,.0f})")
print(f"\nValid (ded, hb, ht, ml) tuples per worker: {n_valid:,}")
print(f"Scan grid: {len(scan_grid)} (revenue, lr, inflection) combos")
print(f"Total evaluations: ~{n_valid * len(scan_grid):,}")

Deductible grid:   35 values ($10,000 -- $228,546,386)
Hollow grid:       30 values ($106,082 -- $554,102,033)
Max limit grid:    26 values ($623,551 -- $1,000,000,000)
Revenue grid:      3 values ($5,000,000 -- $50,000,000)

Valid (ded, hb, ht, ml) tuples per worker: 87,507
Scan grid: 99 (revenue, lr, inflection) combos
Total evaluations: ~8,663,193


In [ ]:
import os
import pickle

from tqdm.auto import tqdm

CACHE_DIR = 'cache'
os.makedirs(CACHE_DIR, exist_ok=True)

# Versioned cache names (v2 = Sobol + IS + Sharpe + min-adequate)
records_base = os.path.join(CACHE_DIR, 'hollow_tower_experiment')
optimal_base = os.path.join(CACHE_DIR, 'hollow_optimal_towers')

N_CORES = multiprocessing.cpu_count()

# --- Optimization criteria ---
RUIN_LIMIT       = 0.0025      # hard constraint: P(ruin) <= 0.25%
SENS_N_PATHS     = 100_000     # main Sobol pool size
IS_N_PATHS       = 2_000       # importance-sampled tail paths
IS_THRESHOLD     = 50_000_000  # tail classification threshold ($50M)
SHARPE_TOLERANCE = 0.005       # 0.5% tolerance for min-adequate selection

# --- Test override ---
# SENS_N_PATHS     = 500     # main Sobol pool size
# IS_N_PATHS       = 20       # importance-sampled tail paths

warnings.filterwarnings("ignore")
logging.getLogger("ergodic_insurance").setLevel(logging.ERROR)


# --- Helper: split a layer region around a hollow section ---
def _get_covered_regions(eff_attach, eff_ceiling, hollow_bottom, hollow_top):
    """Return sub-regions of [eff_attach, eff_ceiling] after removing the hollow."""
    if hollow_top <= eff_attach or hollow_bottom >= eff_ceiling:
        return [(eff_attach, eff_ceiling)]
    if hollow_bottom <= eff_attach and hollow_top >= eff_ceiling:
        return []
    if hollow_bottom > eff_attach and hollow_top < eff_ceiling:
        return [(eff_attach, hollow_bottom), (hollow_top, eff_ceiling)]
    if hollow_bottom <= eff_attach:
        return [(hollow_top, eff_ceiling)]
    return [(eff_attach, hollow_bottom)]


# --- Hollow-tower make_program (for premium computation) ---
def make_program(ded,
                base_loss_ratio: float,
                loss_ratio_inflection: float,
                max_limit=None,
                hollow_bottom=None,
                hollow_top=None,
                pricers=None):
    """Build multi-layer tower with optional hollow (gap) section.

    Coverage bands:
        [ded, hollow_bottom]  +  [hollow_top, max_limit]
    Gap (hollow):
        [hollow_bottom, hollow_top]  -- no coverage, retained by the company.
    """
    layer_defs = [
        (0, 5_000_000, 0),
        (5_000_000, 10_000_000, 0),
        (10_000_000, 25_000_000, 0),
        (25_000_000, 50_000_000, 0),
        (50_000_000, 100_000_000, 0),
        (100_000_000, 150_000_000, 0),
        (150_000_000, 200_000_000, 0),
        (200_000_000, 250_000_000, 0),
        (250_000_000, 300_000_000, 0),
        (300_000_000, 350_000_000, 0),
        (350_000_000, 400_000_000, 0),
        (400_000_000, 450_000_000, 0),
        (450_000_000, 500_000_000, 0),
    ]
    layers = []
    for attach, ceiling, reinst in layer_defs:
        if ded >= ceiling:
            continue
        if max_limit is not None and max_limit <= attach:
            continue
        eff_attach = max(attach, ded)
        eff_ceiling = min(ceiling, max_limit) if max_limit is not None else ceiling
        if eff_attach >= eff_ceiling:
            continue
        if hollow_bottom is not None and hollow_top is not None:
            regions = _get_covered_regions(eff_attach, eff_ceiling,
                                           hollow_bottom, hollow_top)
        else:
            regions = [(eff_attach, eff_ceiling)]
        for r_attach, r_ceiling in regions:
            limit = r_ceiling - r_attach
            if limit <= 0:
                continue
            rol = analytical_rate_on_line(r_attach, limit,
                                          base_loss_ratio,
                                          loss_ratio_inflection, pricers)
            layers.append(EnhancedInsuranceLayer(
                attachment_point=r_attach,
                limit=limit,
                base_premium_rate=rol,
                reinstatements=reinst,
            ))
    name = f"Hollow Tower Ded=${ded:,.0f}"
    if hollow_bottom is not None:
        name += f" Gap=[${hollow_bottom:,.0f}-${hollow_top:,.0f}]"
    if max_limit is not None:
        name += f" Lim=${max_limit:,.0f}"
    return InsuranceProgram(layers=layers, deductible=ded, name=name)


def _optimize_tower_for_revenue(args):
    """Sweep all valid (ded, hb, ht, max_limit) combos for one (revenue, lr, inflection).

    Methodology:
    - Sobol quasi-random sequences (SENS_N_PATHS paths) for main estimation
    - Importance sampling via rejection (IS_N_PATHS tail paths above IS_THRESHOLD)
    - Stratified estimator combining main + IS pools for growth/vol/ruin
    - Vectorized retained-loss computation (analytical, bypasses process_claim)
    """
    revenue, base_lr, inf_lr, ded_arr, hollow_arr, ml_arr, seed_base = args
    initial_assets = revenue / ATR

    # --- Generate main Sobol pool ---
    main_shocks, main_losses, main_max_loss = generate_sobol_loss_pool(
        SENS_N_PATHS, revenue, seed=seed_base)
    tail_mask = main_max_loss > IS_THRESHOLD

    # --- Generate IS pool via rejection ---
    is_shocks, is_losses, _p_tail = generate_is_pool_rejection(
        IS_N_PATHS, revenue, threshold=IS_THRESHOLD,
        seed_offset=seed_base + 1_000_000, verbose=False)

    pricers = make_layer_pricers(cur_revenue=revenue)

    # --- Pre-compute path-level quantities (main pool, once per worker) ---
    rev_m = initial_assets * ATR * np.exp(
        REV_VOL * main_shocks[:, 0] - 0.5 * REV_VOL**2)
    oi_m  = rev_m * OPERATING_MARGIN
    sc_m  = (rev_m / revenue) ** FREQ_SCALING_EXPONENT
    sl_m  = main_losses * sc_m[:, None]   # (SENS_N_PATHS, MAX_EVENTS)

    # --- Pre-compute path-level quantities (IS pool) ---
    rev_i = initial_assets * ATR * np.exp(
        REV_VOL * is_shocks[:, 0] - 0.5 * REV_VOL**2)
    oi_i  = rev_i * OPERATING_MARGIN
    sc_i  = (rev_i / revenue) ** FREQ_SCALING_EXPONENT
    sl_i  = is_losses * sc_i[:, None]     # (IS_N_PATHS, MAX_EVENTS)

    n_events = sl_m.shape[1]
    records = []

    for ded in ded_arr:
        for i_hb, hb in enumerate(hollow_arr):
            if hb < ded:
                continue
            for ht in hollow_arr[i_hb + 1:]:
                for ml in ml_arr:
                    if ml < ht:
                        continue

                    # Premium via existing pricing infrastructure
                    program = make_program(ded, base_lr, inf_lr,
                                           max_limit=ml, hollow_bottom=hb,
                                           hollow_top=ht, pricers=pricers)
                    premium = program.calculate_premium()

                    # --- Vectorized retained loss (main pool) ---
                    # retained = loss - band1_covered - band2_covered
                    # band1 covers [ded, hb], band2 covers [ht, ml]
                    ret_m = np.zeros(SENS_N_PATHS)
                    for j in range(n_events):
                        x = sl_m[:, j]
                        cov = (np.maximum(0, np.minimum(x, hb) - ded) +
                               np.maximum(0, np.minimum(x, ml) - ht))
                        ret_m += x - cov
                    W_m = np.maximum(initial_assets + oi_m - ret_m - premium, 0.0)

                    # --- Vectorized retained loss (IS pool) ---
                    ret_i = np.zeros(IS_N_PATHS)
                    for j in range(n_events):
                        x = sl_i[:, j]
                        cov = (np.maximum(0, np.minimum(x, hb) - ded) +
                               np.maximum(0, np.minimum(x, ml) - ht))
                        ret_i += x - cov
                    W_i = np.maximum(initial_assets + oi_i - ret_i - premium, 0.0)

                    # --- Stratified statistics ---
                    growth_rate, growth_vol, ruin_prob = compute_stratified_stats(
                        W_m, W_i, tail_mask, initial_assets)

                    records.append({
                        'revenue': revenue,
                        'base_loss_ratio': base_lr,
                        'loss_ratio_inflation_factor': inf_lr,
                        'initial_assets': initial_assets,
                        'ded': ded,
                        'hollow_bottom': hb,
                        'hollow_top': ht,
                        'max_limit': ml,
                        'growth_rate': growth_rate,
                        'growth_vol': growth_vol,
                        'ruin_prob': ruin_prob,
                        'premium': premium,
                        'mean_wealth': float(np.mean(W_m)),
                        'covered_width': (hb - ded) + (ml - ht),
                    })

    return records


# === Load from cache or compute ===
try:
    df_tower = pd.read_parquet(records_base + '.parquet')
    print(f"Loaded: {records_base}.parquet ({len(df_tower):,} rows)")
    df_optimal = pd.read_parquet(optimal_base + '.parquet')
    print(f"Loaded: {optimal_base}.parquet ({len(df_optimal)} rows)")
except Exception:
    print(f"Cache miss -- computing hollow tower experiment "
          f"(Sobol + IS + Sharpe + min-adequate)...\n")

    inputs = [(rev, base_lr, inf_lr, ded_grid, hollow_grid, max_limit_grid, SEED + i)
              for i, (rev, base_lr, inf_lr) in enumerate(scan_grid)]
    print(f"Hollow tower optimization: {len(scan_grid)} combos x "
          f"~{n_valid:,} valid (ded, hb, ht, ml) tuples")
    print(f"  Main pool:  {SENS_N_PATHS:,} Sobol paths per worker")
    print(f"  IS pool:    {IS_N_PATHS:,} tail paths (threshold ${IS_THRESHOLD:,.0f})")
    print(f"  Scoring:    Sharpe ratio (growth_rate / growth_vol)")
    print(f"  Constraint: P(ruin) <= {RUIN_LIMIT:.2%}")
    print(f"  Selection:  min-adequate (tolerance {SHARPE_TOLERANCE:.1%} on covered_width)")
    print(f"Parallel on {N_CORES} cores\n")

    t0 = time.time()
    try:
        from joblib import Parallel, delayed
        results = Parallel(n_jobs=min(N_CORES, len(scan_grid)), verbose=10)(
            delayed(_optimize_tower_for_revenue)(inp) for inp in inputs
        )
    except ImportError:
        print("joblib not available, trying ProcessPoolExecutor...")
        try:
            with ProcessPoolExecutor(max_workers=min(N_CORES, len(scan_grid))) as ex:
                results = list(tqdm(
                    ex.map(_optimize_tower_for_revenue, inputs),
                    total=len(inputs), desc="Hollow tower optimization"))
        except Exception:
            print("ProcessPoolExecutor failed, running serially...")
            results = [_optimize_tower_for_revenue(inp)
                       for inp in tqdm(inputs, desc="Hollow tower (serial)")]
    except Exception as e:
        print(f"Parallel failed ({e}), running serially...")
        results = [_optimize_tower_for_revenue(inp)
                   for inp in tqdm(inputs, desc="Hollow tower (serial)")]

    all_records = []
    for r in results:
        all_records.extend(r)

    elapsed = time.time() - t0
    print(f"\nDone in {elapsed:.1f}s ({len(all_records):,} evaluations)")

    df_tower = pd.DataFrame(all_records)

    # --- Select optimal hollow tower per (revenue, base_loss_ratio, inflection) ---
    # Scoring:   Sharpe = growth_rate / growth_vol
    # Selection: min-adequate = smallest covered_width within SHARPE_TOLERANCE of best
    optimal_towers = []
    for rev, base_lr, inf_lr in scan_grid:
        mask = ((df_tower['revenue'] == rev) &
                (np.isclose(df_tower['base_loss_ratio'], base_lr)) &
                (np.isclose(df_tower['loss_ratio_inflation_factor'], inf_lr)))
        df_combo = df_tower[mask]
        feasible = df_combo[df_combo['ruin_prob'] <= RUIN_LIMIT]

        if len(feasible) > 0:
            g = feasible['growth_rate'].values
            v = feasible['growth_vol'].values
            sharpe = np.where(v > 1e-12, g / v, np.sign(g) * 1e12)
            best_sharpe = np.max(sharpe)

            # Near-optimal: within SHARPE_TOLERANCE of best
            threshold_val = best_sharpe - abs(best_sharpe) * SHARPE_TOLERANCE
            near_opt_mask = sharpe >= threshold_val
            near_opt = feasible[near_opt_mask].copy()
            near_opt['_sharpe'] = sharpe[near_opt_mask]

            # Min-adequate: smallest covered_width, then lowest max_limit
            best_idx = near_opt.sort_values(
                ['covered_width', 'max_limit']).index[0]
        else:
            best_idx = df_combo['ruin_prob'].idxmin()

        row = df_tower.loc[best_idx]
        g_val = row['growth_rate']
        v_val = row['growth_vol']
        sharpe_val = g_val / v_val if v_val > 1e-12 else np.sign(g_val) * 1e12

        covered_width = (row['hollow_bottom'] - row['ded']) + \
                        (row['max_limit'] - row['hollow_top'])
        optimal_towers.append({
            'revenue': rev,
            'base_loss_ratio': base_lr,
            'loss_ratio_inflation_factor': inf_lr,
            'initial_assets': row['initial_assets'],
            'optimal_ded': row['ded'],
            'optimal_hollow_bottom': row['hollow_bottom'],
            'optimal_hollow_top': row['hollow_top'],
            'optimal_max_limit': row['max_limit'],
            'growth_rate': g_val,
            'growth_vol': v_val,
            'sharpe_ratio': sharpe_val,
            'ruin_prob': row['ruin_prob'],
            'premium': row['premium'],
            'covered_width': covered_width,
            'hollow_width': row['hollow_top'] - row['hollow_bottom'],
        })

    df_optimal = pd.DataFrame(optimal_towers)

    print(f"\nOptimal hollow towers: {len(df_optimal)} combos "
          f"({len(revenue_grid)} revenues x {len(base_loss_ratio_grid)} LRs "
          f"x {len(inflection_factor_grid)} inflections)")
    print(df_optimal[['revenue', 'base_loss_ratio', 'loss_ratio_inflation_factor',
                       'optimal_ded', 'optimal_hollow_bottom', 'optimal_hollow_top',
                       'optimal_max_limit', 'sharpe_ratio', 'ruin_prob']].head(20).to_string())

    # --- Cache Results ---
    try:
        df_tower.to_parquet(records_base + '.parquet', compression='zstd')
        sz = os.path.getsize(records_base + '.parquet') / 1024
        print(f"\nSaved: {records_base}.parquet ({len(df_tower):,} rows, {sz:.0f} KB)")
    except Exception as e:
        print(f"Parquet save failed: {e}")

    df_tower.to_pickle(records_base + '.pkl')
    print(f"Saved: {records_base}.pkl ({len(df_tower):,} rows)")

    try:
        df_optimal.to_parquet(optimal_base + '.parquet', compression='zstd')
        print(f"Saved: {optimal_base}.parquet ({len(df_optimal)} rows)")
    except Exception as e:
        print(f"Parquet save failed: {e}")

    df_optimal.to_pickle(optimal_base + '.pkl')
    print(f"Saved: {optimal_base}.pkl ({len(df_optimal)} rows)")

    print(f"\nCache directory: {os.path.abspath(CACHE_DIR)}")

Cache miss -- computing hollow tower experiment (Sobol + IS + Sharpe + min-adequate)...

Hollow tower optimization: 99 combos x ~87,507 valid (ded, hb, ht, ml) tuples
  Main pool:  100,000 Sobol paths per worker
  IS pool:    2,000 tail paths (threshold $50,000,000)
  Scoring:    Sharpe ratio (growth_rate / growth_vol)
  Constraint: P(ruin) <= 0.25%
  Selection:  min-adequate (tolerance 0.5% on covered_width)
Parallel on 44 cores



[Parallel(n_jobs=44)]: Using backend LokyBackend with 44 concurrent workers.


### Hollow Tower Demand Curve Visualization (3x3 Facet Plot)

Each facet shows how the **optimal hollow tower** varies with the base loss ratio (x-axis):

- **Columns** = starting revenue (\$5M, \$25M, \$50M) — revenue increases left → right
- **Rows** = excess-layer inflection factor (1×, 2×, 3×) — excess pricing increases bottom → top
- **Blue line** = optimal max limit (top of tower)
- **Orange line** = optimal deductible (retention / bottom of tower)
- **Gray lines + shading** = hollow section (gap in coverage — losses here are fully retained)

The hollow section represents a deliberate gap where the company self-insures,
buying primary coverage below and excess coverage above.  As the base loss ratio
rises (insurance becomes cheaper), the optimal hollow section should narrow or shift.

In [ ]:
df_optimal_old = df_optimal.copy(deep=True)

for idx, row in df_optimal.iterrows():
  if row['optimal_hollow_bottom'] <= row['optimal_ded']:
    df_optimal.at[idx, 'optimal_hollow_bottom'] = row['optimal_hollow_top']
    df_optimal.at[idx, 'optimal_ded'] = row['optimal_hollow_top']
    # print(row)
  if row['optimal_hollow_top'] >= row['optimal_max_limit']:
    df_optimal.at[idx, 'optimal_hollow_top'] = row['optimal_hollow_bottom']
    df_optimal.at[idx, 'optimal_max_limit'] = row['optimal_hollow_bottom']
    # print(row)

In [ ]:
from scipy.interpolate import PchipInterpolator
import matplotlib.ticker as mticker

def dollar_fmt(x, pos):
    if x >= 1e9:
        return f'${x/1e9:.0f}B'
    elif x >= 1e6:
        return f'${x/1e6:.0f}M'
    elif x >= 1e3:
        return f'${x/1e3:.0f}K'
    return f'${x:.0f}'

def tick_dollar_fmt(x):
    if x >= 1e6:
        return f'${x/1e6:.0f}M'
    elif x >= 1e3:
        return f'${x/1e3:.0f}K'
    return f'${x:.0f}'

# Sort grids: columns = revenue ascending, rows = inflection descending
rev_sorted = sorted(revenue_grid)
inf_sorted = sorted(inflection_factor_grid, reverse=True)

n_rows = len(inf_sorted)
n_cols = len(rev_sorted)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 3 * n_rows),
                         dpi=150, sharex=True, sharey=False)

# Ensure axes is always 2D
if n_rows == 1 and n_cols == 1:
    axes = np.array([[axes]])
elif n_rows == 1:
    axes = axes[np.newaxis, :]
elif n_cols == 1:
    axes = axes[:, np.newaxis]

# Common y-axis bounds across all facets
all_ded = df_optimal['optimal_ded'].values
all_lim = df_optimal['optimal_max_limit'].values
pos_ded = all_ded[all_ded > 0]
y_min = max(pos_ded.min() * 0.5, 1_000) if len(pos_ded) > 0 else 1_000
y_max = all_lim.max() * 2.0

y_ticks = [100_000, 1_000_000, 10_000_000, 100_000_000, 500_000_000]

for row_idx, inf_f in enumerate(inf_sorted):
    for col_idx, rev in enumerate(rev_sorted):
        ax = axes[row_idx, col_idx]

        # Filter data for this facet
        mask = (np.isclose(df_optimal['loss_ratio_inflation_factor'], inf_f)) & \
               (df_optimal['revenue'] == rev)
        df_f = df_optimal[mask].sort_values('base_loss_ratio')

        if len(df_f) < 2:
            ax.text(0.5, 0.5, 'Insufficient data', transform=ax.transAxes,
                    ha='center', va='center', fontsize=9, color='gray')
            continue

        x = df_f['base_loss_ratio'].values
        y_ded = np.maximum(df_f['optimal_ded'].values, 1_000)  # floor for log scale
        y_hb  = np.maximum(df_f['optimal_hollow_bottom'].values, 1_000)
        y_ht  = df_f['optimal_hollow_top'].values
        y_lim = df_f['optimal_max_limit'].values

        # PCHIP interpolation in log-space for smooth curves
        x_fine = np.linspace(x.min(), x.max(), 200)

        if len(x) >= 4:
            ded_interp = PchipInterpolator(x, np.log10(y_ded))
            hb_interp  = PchipInterpolator(x, np.log10(y_hb))
            ht_interp  = PchipInterpolator(x, np.log10(y_ht))
            lim_interp = PchipInterpolator(x, np.log10(y_lim))
            ded_smooth = 10 ** ded_interp(x_fine)
            hb_smooth  = 10 ** hb_interp(x_fine)
            ht_smooth  = 10 ** ht_interp(x_fine)
            lim_smooth = 10 ** lim_interp(x_fine)
        else:
            ded_smooth = 10 ** np.interp(x_fine, x, np.log10(y_ded))
            hb_smooth  = 10 ** np.interp(x_fine, x, np.log10(y_hb))
            ht_smooth  = 10 ** np.interp(x_fine, x, np.log10(y_ht))
            lim_smooth = 10 ** np.interp(x_fine, x, np.log10(y_lim))

        # Hollow section — gray lines with shaded gap
        ax.plot(x_fine, hb_smooth, color='gray', linewidth=1.5, linestyle='--')
        ax.plot(x_fine, ht_smooth, color='gray', linewidth=1.5, linestyle='--')
        ax.fill_between(x_fine, hb_smooth, ht_smooth,
                        alpha=0.25, color='gray', label='Hollow (no coverage)')

        # Plot smooth lines — max limit (blue) and deductible (orange)
        ax.plot(x_fine, lim_smooth, color='steelblue', linewidth=2, label='Max Limit')
        ax.plot(x_fine, ded_smooth, color='darkorange', linewidth=2, label='Deductible')

        # Plot actual data points
        ax.scatter(x, y_lim, color='steelblue', s=20, zorder=5, alpha=0.7)
        ax.scatter(x, y_ded, color='darkorange', s=20, zorder=5, alpha=0.7)
        ax.scatter(x, y_hb, color='gray', s=12, zorder=5, alpha=0.5, marker='v')
        ax.scatter(x, y_ht, color='gray', s=12, zorder=5, alpha=0.5, marker='^')

        ax.set_yscale('log')
        ax.set_ylim(y_min, y_max)
        ax.grid(True, alpha=0.3, which='both')

        ax.yaxis.set_major_formatter(mticker.FuncFormatter(dollar_fmt))
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([tick_dollar_fmt(v) for v in y_ticks])

        for pos in y_ticks:
            ax.axhline(y=pos, color='black', linestyle='dashed', alpha=0.4)

        # Column titles (top row only)
        if row_idx == 0:
            ax.set_title(f'Revenue = ${rev/1e6:.0f}M', fontsize=14, fontweight='bold')

        # Row labels (left column only)
        if col_idx == 0:
            ax.set_ylabel(f'Inflection = {inf_f:.0f}x\nDemanded Amount ($)', fontsize=14)

        # X-axis label (bottom row only)
        if row_idx == n_rows - 1:
            ax.set_xlabel('Base Loss Ratio', fontsize=14)

# Shared legend from first facet
handles, labels = axes[0, 0].get_legend_handles_labels()
if handles:
    fig.legend(handles, labels, loc='upper center', ncol=4, fontsize=12,
               bbox_to_anchor=(0.5, 0.99), frameon=True, edgecolor='black')

fig.suptitle('Hollow Tower Demand Curves: Optimal Retention, Gap & Limit vs. Base Loss Ratio\n'
             f'Sharpe ratio optimization, \u22640.25% ruin constraint, {SENS_N_PATHS} paths',
             fontsize=14, fontweight='bold', y=1.03)

fig.patch.set_facecolor('white')
for row in axes:
    for ax in row:
        ax.set_facecolor('white')
        for spine in ax.spines.values():
            spine.set_color('black')
        ax.tick_params(colors='black')

plt.tight_layout(rect=[0, 0, 1, 0.96], h_pad=4.0, w_pad=4.0)
plt.savefig(os.path.join(CACHE_DIR, 'hollow_demand_cur7ve_3x3.png'),
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f"\nChart saved to: {os.path.join(CACHE_DIR, 'hollow_demand_curve_3x3.png')}")

In [ ]:
from scipy.interpolate import PchipInterpolator
import matplotlib.ticker as mticker

def dollar_fmt(x, pos):
    if x >= 1e9:
        return f'${x/1e9:.0f}B'
    elif x >= 1e6:
        return f'${x/1e6:.0f}M'
    elif x >= 1e3:
        return f'${x/1e3:.0f}K'
    return f'${x:.0f}'

def tick_dollar_fmt(x):
    if x >= 1e6:
        return f'${x/1e6:.0f}M'
    elif x >= 1e3:
        return f'${x/1e3:.0f}K'
    return f'${x:.0f}'

fig, ax = plt.subplots(figsize=(10, 6), dpi=150)

# Common y-axis bounds
all_ded = df_optimal['optimal_ded'].values
all_lim = df_optimal['optimal_max_limit'].values
pos_ded = all_ded[all_ded > 0]
y_min = max(pos_ded.min() * 0.5, 1_000) if len(pos_ded) > 0 else 1_000
y_max = all_lim.max() * 2.0

y_ticks = [100_000, 1_000_000, 10_000_000, 100_000_000, 500_000_000]

inf_f = inf_sorted[-1]   # 1.0x (cheapest excess)
rev = rev_sorted[0]       # $5M (smallest revenue)

# Filter data for this facet
mask = (np.isclose(df_optimal['loss_ratio_inflation_factor'], inf_f)) & \
        (df_optimal['revenue'] == rev)
df_f = df_optimal[mask].sort_values('base_loss_ratio')

x = df_f['base_loss_ratio'].values
y_ded = np.maximum(df_f['optimal_ded'].values, 1_000)  # floor for log scale
y_hb  = np.maximum(df_f['optimal_hollow_bottom'].values, 1_000)
y_ht  = df_f['optimal_hollow_top'].values
y_lim = df_f['optimal_max_limit'].values

# PCHIP interpolation in log-space for smooth curves
x_fine = np.linspace(x.min(), x.max(), 200)

if len(x) >= 4:
    ded_interp = PchipInterpolator(x, np.log10(y_ded))
    hb_interp  = PchipInterpolator(x, np.log10(y_hb))
    ht_interp  = PchipInterpolator(x, np.log10(y_ht))
    lim_interp = PchipInterpolator(x, np.log10(y_lim))
    ded_smooth = 10 ** ded_interp(x_fine)
    hb_smooth  = 10 ** hb_interp(x_fine)
    ht_smooth  = 10 ** ht_interp(x_fine)
    lim_smooth = 10 ** lim_interp(x_fine)
else:
    ded_smooth = 10 ** np.interp(x_fine, x, np.log10(y_ded))
    hb_smooth  = 10 ** np.interp(x_fine, x, np.log10(y_hb))
    ht_smooth  = 10 ** np.interp(x_fine, x, np.log10(y_ht))
    lim_smooth = 10 ** np.interp(x_fine, x, np.log10(y_lim))

# Hollow section — gray lines with shaded gap
ax.plot(x_fine, hb_smooth, color='gray', linewidth=1.5, linestyle='--')
ax.plot(x_fine, ht_smooth, color='gray', linewidth=1.5, linestyle='--')
ax.fill_between(x_fine, hb_smooth, ht_smooth,
                alpha=0.25, color='gray', label='Hollow (no coverage)')

# Plot smooth lines — max limit (blue) and deductible (orange)
ax.plot(x_fine, lim_smooth, color='steelblue', linewidth=2, label='Max Limit')
ax.plot(x_fine, ded_smooth, color='darkorange', linewidth=2, label='Deductible')

# Plot actual data points
ax.scatter(x, y_lim, color='steelblue', s=20, zorder=5, alpha=0.7)
ax.scatter(x, y_ded, color='darkorange', s=20, zorder=5, alpha=0.7)
ax.scatter(x, y_hb, color='gray', s=15, zorder=5, alpha=0.5, marker='v')
ax.scatter(x, y_ht, color='gray', s=15, zorder=5, alpha=0.5, marker='^')

ax.set_yscale('log')
ax.set_ylim(y_min, y_max)
ax.grid(True, alpha=0.3, which='both')

ax.yaxis.set_major_formatter(mticker.FuncFormatter(dollar_fmt))
ax.set_yticks(y_ticks)
ax.set_yticklabels([tick_dollar_fmt(v) for v in y_ticks])

for pos in y_ticks:
    ax.axhline(y=pos, color='black', linestyle='dashed', alpha=0.4)

ax.legend(loc='best', fontsize=12, frameon=True, edgecolor='black')

fig.suptitle('Hollow Tower Demand Curve: Optimal Retention, Gap & Limit vs. Loss Ratio\n'
             f'Inflection = {inf_f:.0f}x, revenue = ${rev/1e6:.0f}M, {SENS_N_PATHS} paths',
             fontsize=14, fontweight='bold', y=1.08)

fig.patch.set_facecolor('white')
ax.set_facecolor('white')
for spine in ax.spines.values():
    spine.set_color('black')
ax.tick_params(colors='black')

ax.set_xlabel('Loss Ratio', fontsize=14)
ax.set_ylabel('Demanded Amount ($)', fontsize=14)

plt.tight_layout()
plt.savefig(os.path.join(CACHE_DIR, 'hollow_demand_curve_base.png'),
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print(f"\nChart saved to: {os.path.join(CACHE_DIR, 'hollow_demand_curve_base.png')}")

# Summary table
print(f"\nBase case summary (inflection={inf_f:.0f}x, revenue=${rev/1e6:.0f}M):")
print(f"  {'LR':>6s}  {'Ded':>10s}  {'Hollow Bot':>12s}  {'Hollow Top':>12s}  "
      f"{'Max Limit':>12s}  {'Premium':>10s}  {'Growth':>8s}")
print(f"  {'------':>6s}  {'--------':>10s}  {'----------':>12s}  {'----------':>12s}  "
      f"{'----------':>12s}  {'--------':>10s}  {'------':>8s}")
for _, row in df_f.iterrows():
    print(f"  {row['base_loss_ratio']:>6.2f}"
          f"  ${row['optimal_ded']:>9,.0f}"
          f"  ${row['optimal_hollow_bottom']:>11,.0f}"
          f"  ${row['optimal_hollow_top']:>11,.0f}"
          f"  ${row['optimal_max_limit']:>11,.0f}"
          f"  ${row['premium']:>9,.0f}"
          f"  {row['growth_rate']:>+7.1%}")

### Demand Elasticity Analysis

The **price elasticity of demand** measures how responsive the optimal hollow tower is
to changes in market pricing.  We compute:

$$E = \frac{d(\ln Q)}{d(\ln \text{LR})}$$

where $Q$ is the demanded amount (deductible, hollow boundaries, or max limit) and LR is the base loss ratio.

- $|E| > 1$: **Elastic** — demand responds more than proportionally to price changes
- $|E| < 1$: **Inelastic** — demand responds less than proportionally
- $E > 0$ for max limit: cheaper insurance → buy more coverage on top
- $E < 0$ for deductible: cheaper insurance → lower retention (buy more from the bottom)

In [ ]:
## --- Demand Elasticity Chart (Base Case) ---

from scipy.interpolate import PchipInterpolator
import matplotlib.ticker as mticker

# Re-extract base case data (same filter as cell-16)
inf_f = inf_sorted[-1]   # 1.0x (cheapest excess)
rev = rev_sorted[0]       # $5M (smallest revenue)

mask = (np.isclose(df_optimal['loss_ratio_inflation_factor'], inf_f)) & \
       (df_optimal['revenue'] == rev)
df_f = df_optimal[mask].sort_values('base_loss_ratio')

x = df_f['base_loss_ratio'].values
y_ded = np.maximum(df_f['optimal_ded'].values, 1_000)
y_hb  = np.maximum(df_f['optimal_hollow_bottom'].values, 1_000)
y_ht  = df_f['optimal_hollow_top'].values
y_lim = df_f['optimal_max_limit'].values

# PCHIP in log-space
ded_interp = PchipInterpolator(x, np.log10(y_ded))
hb_interp  = PchipInterpolator(x, np.log10(y_hb))
ht_interp  = PchipInterpolator(x, np.log10(y_ht))
lim_interp = PchipInterpolator(x, np.log10(y_lim))

x_fine = np.linspace(x.min(), x.max(), 200)

# Elasticity: E = LR * ln(10) * d(log10 Q) / d(LR)
ded_d1 = ded_interp.derivative()(x_fine)
hb_d1  = hb_interp.derivative()(x_fine)
ht_d1  = ht_interp.derivative()(x_fine)
lim_d1 = lim_interp.derivative()(x_fine)

ded_elast = x_fine * np.log(10) * ded_d1
hb_elast  = x_fine * np.log(10) * hb_d1
ht_elast  = x_fine * np.log(10) * ht_d1
lim_elast = x_fine * np.log(10) * lim_d1

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 6), dpi=150)

ax.plot(x_fine, lim_elast, color='#2ca02c', linewidth=2.5, linestyle='-',
        label='Max Limit elasticity')
ax.plot(x_fine, ded_elast, color='#d62728', linewidth=2.5, linestyle='--',
        label='Deductible elasticity')
ax.plot(x_fine, hb_elast, color='gray', linewidth=1.5, linestyle='-.',
        label='Hollow Bottom elasticity', alpha=0.7)
ax.plot(x_fine, ht_elast, color='dimgray', linewidth=1.5, linestyle=':',
        label='Hollow Top elasticity', alpha=0.7)

# Reference lines
ax.axhline(y=0, color='black', linewidth=1, alpha=0.8)
ax.axhline(y=1, color='gray', linewidth=1, linestyle=':', alpha=0.6,
           label='Unit elasticity ($|E|=1$)')
ax.axhline(y=-1, color='gray', linewidth=1, linestyle=':', alpha=0.6)

# Shade elastic vs inelastic regions
ax.axhspan(-1, 1, color='lightyellow', alpha=0.3, zorder=0)
ax.text(x.min() + 0.01, 0.5, 'Inelastic\n$|E| < 1$',
        fontsize=10, color='gray', alpha=0.7, va='center')

ax.set_xlabel('Base Loss Ratio', fontsize=14)
ax.set_ylabel('Price Elasticity of Demand  $E$', fontsize=14)
ax.legend(loc='best', fontsize=10, frameon=True, edgecolor='black')
ax.grid(True, alpha=0.3)

fig.suptitle('Hollow Tower Demand Elasticity: How Responsive Is the Optimal Tower to Price?\n'
             f'Base case: inflection = {inf_f:.0f}x, revenue = ${rev/1e6:.0f}M, '
             f'{SENS_N_PATHS:,} paths',
             fontsize=13, fontweight='bold', y=1.06)

fig.patch.set_facecolor('white')
ax.set_facecolor('white')
for spine in ax.spines.values():
    spine.set_color('black')
ax.tick_params(colors='black')

plt.tight_layout()
plt.savefig(os.path.join(CACHE_DIR, 'hollow_demand_elasticity_base.png'),
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()
print(f"\nChart saved to: {os.path.join(CACHE_DIR, 'hollow_demand_elasticity_base.png')}")

# Print elasticity at key loss ratios
print("\nElasticity at key loss ratios:")
print(f"  {'LR':>6s}  {'E(Limit)':>10s}  {'E(Ded)':>10s}  {'E(HB)':>10s}  {'E(HT)':>10s}  {'Interpretation'}")
print(f"  {'------':>6s}  {'--------':>10s}  {'------':>10s}  {'------':>10s}  {'------':>10s}  {'-' * 20}")
for lr_check in [0.45, 0.55, 0.65, 0.70, 0.80]:
    idx = np.argmin(np.abs(x_fine - lr_check))
    e_lim = lim_elast[idx]
    e_ded = ded_elast[idx]
    e_hb  = hb_elast[idx]
    e_ht  = ht_elast[idx]
    any_elastic = abs(e_lim) > 1 or abs(e_ded) > 1 or abs(e_hb) > 1 or abs(e_ht) > 1
    interp = "Elastic" if any_elastic else "Inelastic"
    print(f"  {lr_check:>6.2f}  {e_lim:>+10.2f}  {e_ded:>+10.2f}  {e_hb:>+10.2f}  {e_ht:>+10.2f}  {interp}")

### Hollow Tower Coverage Bands & Premium Efficiency

Two additional views of the base case hollow tower demand curve:

1. **Coverage Bands** — Two shaded regions show the lower and upper coverage bands,
   with the hollow gap visible between them.  The covered width (sum of both bands)
   as a fraction of revenue reveals how much exposure is transferred.

2. **Premium vs. Growth Rate** — Shows the cost-benefit tradeoff: how much premium
   the company pays at each loss ratio, and what ergodic growth rate it achieves.
   The hollow tower should achieve comparable growth at lower premium by dropping
   coverage in the mid-layer where premium-per-unit-of-protection is least efficient.

In [ ]:
## --- Chart: Hollow Tower Coverage Bands + Premium vs Growth (base case) ---

from scipy.interpolate import PchipInterpolator
import matplotlib.ticker as mticker

def dollar_fmt(x, pos):
    if x >= 1e9:  return f'${x/1e9:.0f}B'
    elif x >= 1e6: return f'${x/1e6:.0f}M'
    elif x >= 1e3: return f'${x/1e3:.0f}K'
    return f'${x:.0f}'

# Base case filter
inf_f = inf_sorted[-1]   # 1.0x
rev = rev_sorted[0]       # $5M

mask = (np.isclose(df_optimal['loss_ratio_inflation_factor'], inf_f)) & \
       (df_optimal['revenue'] == rev)
df_f = df_optimal[mask].sort_values('base_loss_ratio')

x = df_f['base_loss_ratio'].values
y_ded = np.maximum(df_f['optimal_ded'].values, 1_000)
y_hb  = np.maximum(df_f['optimal_hollow_bottom'].values, 1_000)
y_ht  = df_f['optimal_hollow_top'].values
y_lim = df_f['optimal_max_limit'].values
y_prem = df_f['premium'].values
y_growth = df_f['growth_rate'].values
y_covered = df_f['covered_width'].values
y_hollow = df_f['hollow_width'].values

# PCHIP interpolants
x_fine = np.linspace(x.min(), x.max(), 200)
ded_interp = PchipInterpolator(x, np.log10(y_ded))
hb_interp  = PchipInterpolator(x, np.log10(y_hb))
ht_interp  = PchipInterpolator(x, np.log10(y_ht))
lim_interp = PchipInterpolator(x, np.log10(y_lim))
ded_smooth = 10 ** ded_interp(x_fine)
hb_smooth  = 10 ** hb_interp(x_fine)
ht_smooth  = 10 ** ht_interp(x_fine)
lim_smooth = 10 ** lim_interp(x_fine)

prem_interp = PchipInterpolator(x, y_prem)
prem_smooth = prem_interp(x_fine)

growth_interp = PchipInterpolator(x, y_growth)
growth_smooth = growth_interp(x_fine)

covered_interp = PchipInterpolator(x, np.log10(np.maximum(y_covered, 1)))
covered_smooth = 10 ** covered_interp(x_fine)

# ---- Figure with 2 subplots stacked ----
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10), dpi=150,
                                gridspec_kw={'height_ratios': [1, 1]})

# --- Panel 1: Coverage Bands with Hollow Gap ---
# Lower coverage band (ded to hollow_bottom)
ax1.fill_between(x_fine, ded_smooth, hb_smooth,
                 alpha=0.2, color='steelblue', label='Lower coverage band')
# Upper coverage band (hollow_top to max_limit)
ax1.fill_between(x_fine, ht_smooth, lim_smooth,
                 alpha=0.2, color='darkorange', label='Upper coverage band')
# Hollow gap
ax1.fill_between(x_fine, hb_smooth, ht_smooth,
                 alpha=0.15, color='gray', hatch='///', label='Hollow (no coverage)')

ax1.plot(x_fine, lim_smooth, color='steelblue', linewidth=2, label='Max Limit')
ax1.plot(x_fine, ded_smooth, color='darkorange', linewidth=2, label='Deductible')
ax1.plot(x_fine, hb_smooth, color='gray', linewidth=1.5, linestyle='--')
ax1.plot(x_fine, ht_smooth, color='gray', linewidth=1.5, linestyle='--')

# Covered width on secondary axis
ax1b = ax1.twinx()
ax1b.plot(x_fine, covered_smooth, color='#9467bd', linewidth=2, linestyle='-.',
          label='Covered Width', alpha=0.85)
ax1b.set_ylabel('Covered Width ($)', fontsize=12, color='#9467bd')
ax1b.yaxis.set_major_formatter(mticker.FuncFormatter(dollar_fmt))
ax1b.tick_params(axis='y', colors='#9467bd')
ax1b.set_yscale('log')

ax1.set_yscale('log')
y_ticks = [100_000, 1_000_000, 10_000_000, 100_000_000, 500_000_000]
ax1.set_yticks(y_ticks)
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(dollar_fmt))
ax1.set_ylabel('Optimal Tower Bounds ($)', fontsize=12)
ax1.set_xlabel('Base Loss Ratio', fontsize=12)
ax1.grid(True, alpha=0.3, which='both')

# Combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax1b.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2,
           loc='upper left', fontsize=9, frameon=True, edgecolor='black')

ax1.set_title('Hollow Tower Coverage Bands',
              fontsize=13, fontweight='bold')

# --- Panel 2: Premium vs Growth Rate ---
ax2.plot(x_fine, prem_smooth, color='#8c564b', linewidth=2.5, linestyle='-',
         label='Annual Premium')
ax2.scatter(x, y_prem, color='#8c564b', s=25, zorder=5, alpha=0.7)
ax2.set_ylabel('Annual Premium ($)', fontsize=12, color='#8c564b')
ax2.yaxis.set_major_formatter(mticker.FuncFormatter(dollar_fmt))
ax2.tick_params(axis='y', colors='#8c564b')

# Growth rate on secondary axis
ax2b = ax2.twinx()
ax2b.plot(x_fine, growth_smooth * 100, color='#2ca02c', linewidth=2.5,
          linestyle='--', label='Ergodic Growth Rate')
ax2b.scatter(x, y_growth * 100, color='#2ca02c', s=25, zorder=5, alpha=0.7)
ax2b.set_ylabel('Ergodic Growth Rate (%)', fontsize=12, color='#2ca02c')
ax2b.tick_params(axis='y', colors='#2ca02c')
ax2b.axhline(y=0, color='#2ca02c', linewidth=0.8, linestyle=':', alpha=0.5)

ax2.set_xlabel('Base Loss Ratio', fontsize=12)
ax2.grid(True, alpha=0.3)

# Combined legend
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax2b.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2,
           loc='best', fontsize=10, frameon=True, edgecolor='black')

ax2.set_title('Premium Cost vs. Ergodic Growth Rate',
              fontsize=13, fontweight='bold')

# Style both panels
for ax in [ax1, ax2]:
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_color('black')
    ax.tick_params(colors='black')
for ax in [ax1b, ax2b]:
    for spine in ax.spines.values():
        spine.set_color('black')

fig.patch.set_facecolor('white')
fig.suptitle('Hollow Tower Structure & Economic Impact vs. Market Pricing\n'
             f'Base case: inflection = {inf_f:.0f}x, revenue = ${rev/1e6:.0f}M, '
             f'{SENS_N_PATHS:,} paths',
             fontsize=14, fontweight='bold', y=1.02)

plt.tight_layout()
plt.savefig(os.path.join(CACHE_DIR, 'hollow_coverage_band_and_premium.png'),
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()
print(f"\nChart saved to: {os.path.join(CACHE_DIR, 'hollow_coverage_band_and_premium.png')}")

# Summary statistics
print(f"\nBase case summary:")
print(f"  {'LR':>6s}  {'Ded':>10s}  {'H.Bot':>10s}  {'H.Top':>10s}  "
      f"{'Max Lim':>12s}  {'Covered':>12s}  {'Premium':>10s}  {'Growth':>8s}")
print(f"  {'------':>6s}  {'--------':>10s}  {'--------':>10s}  {'--------':>10s}  "
      f"{'----------':>12s}  {'----------':>12s}  {'--------':>10s}  {'------':>8s}")
for _, row in df_f.iterrows():
    print(f"  {row['base_loss_ratio']:>6.2f}"
          f"  ${row['optimal_ded']:>9,.0f}"
          f"  ${row['optimal_hollow_bottom']:>9,.0f}"
          f"  ${row['optimal_hollow_top']:>9,.0f}"
          f"  ${row['optimal_max_limit']:>11,.0f}"
          f"  ${row['covered_width']:>11,.0f}"
          f"  ${row['premium']:>9,.0f}"
          f"  {row['growth_rate']:>+7.1%}")